# Simple NN with pooling

In [1]:
from sklearn.metrics import f1_score
import pandas as pd
import datautils
import simpleNN
import torch
from sklearn.model_selection import ParameterGrid
import time

In [2]:
PATH = './Dataset/power-gb-train.tsv'
RES_DIR = './Results/'
EMBED_DIR = './Embeddings/'

## Tf-idf embeddings

In [10]:
X_train, y_train, X_val, y_val, _, _ = datautils.split_kfold_dataset(PATH)

for idx, (tr_fold, val_fold) in enumerate(zip(X_train, X_val)):

    X_train[idx], vectorizer = datautils.tf_idf_preprocessing(tr_fold)
    X_val[idx] = vectorizer.transform(val_fold)

In [44]:



grid = list(ParameterGrid(
    {
        "hidden_neurons": [20], 
        "lr": [1e-3, 1e-2, 1e-1],
        "epochs": [10,20],
        "batch size": [1,5,10]
    }
))
# train simple NN here
results_df = pd.DataFrame(
    columns=["hidden neurons","optimizer","batch size" ,"epochs", "F1 Score", "Recall", "Precision"]
)
print(f"Training a simple NN")
for params in grid:
    hidden_neurons = params["hidden_neurons"]
    lr = params["lr"]
    epochs = params["epochs"]
    batch_size = params["batch size"]
    print(f"hyperparams: hidden neurons={hidden_neurons} lr={lr} epochs={epochs} batch size={1}:")
    for k, (fold_X_train,fold_y_train,fold_X_val,fold_y_val) in enumerate(zip(X_train, y_train,X_val,y_val)):
        print("Fold: ", k+1)
        fold_X_train = np.array(fold_X_train)
        fold_X_val = np.array(fold_X_val)
        simple_nn=simpleNN.SimpleNN(fold_X_train.shape[1],hidden_neurons)
        start = time.time()
        fold_precision,fold_recall,fold_fscore=simpleNN.train_simpleNN(
                    fold_X_train,fold_X_val, 
                    fold_y_train.values,fold_y_val.values,
                    loss_fn=torch.nn.BCELoss(),
                    model = simple_nn, 
                    optimizer=torch.optim.SGD(simple_nn.parameters(),lr=lr),
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=True
                    )
        stop = time.time()
        print(f"precision={fold_precision},recall={fold_recall},score={fold_fscore}")
        results_df = pd.concat(
            [
                results_df,
                pd.DataFrame(
                    {
                        "batch size": batch_size,
                        "hidden neurons": params["hidden_neurons"],
                        "learning rate": params["lr"],
                        "optimizer": "SGD",
                        "epochs": params["epochs"],
                        "Fold": k,
                        "Loss":"BCELoss",
                        "F1 Score": fold_fscore,
                        "Recall": fold_recall,
                        "Precision": fold_precision,
                        "Time": stop - start,

                    },
                    index=[0],
                ),
            ],
            ignore_index=True,
        )
    # save results every time we finish a fold in case of crash
    results_df.to_csv(RES_DIR + "results-SimpleNN-tfidf.csv", index=False)
results_df

Training a simple NN
hyperparams: hidden neurons=20 lr=0.001 epochs=10 batch size=1:
Fold:  1
'Epoch 1 -------------------------------'
'loss: 0.444375  [    1/23944]'
'loss: 0.580090  [ 1001/23944]'
'loss: 0.811432  [ 2001/23944]'
'loss: 0.629938  [ 3001/23944]'


KeyboardInterrupt: 

## Word2vec embeddings

In [3]:
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess

In [4]:
X_train, y_train, X_val, y_val, _, _ = datautils.split_kfold_dataset(PATH)

for idx, (tr_fold, val_fold) in enumerate(zip(X_train, X_val)):
    word_vectors = KeyedVectors.load(EMBED_DIR+f'w2v-fold{idx}.kv', mmap='r')

    tr_fold = list(map(simple_preprocess, tr_fold))
    val_fold = list(map(simple_preprocess, val_fold))

    X_train[idx] = datautils.documents_vector_wv(tr_fold,word_vectors)
    X_val[idx] = datautils.documents_vector_wv(val_fold,word_vectors)

In [6]:
# train simple NN here



import numpy as np


grid = list(ParameterGrid(
    {
        "hidden_neurons": [20], 
        "lr": [1e-3, 1e-2, 1e-1],
        "epochs": [10,20],
        "batch size": [1,5,10]
    }
))
# train simple NN here
results_df = pd.DataFrame(
    columns=["hidden neurons","optimizer","batch size" ,"epochs", "F1 Score", "Recall", "Precision"]
)
print(f"Training a simple NN")
for params in grid:
    hidden_neurons = params["hidden_neurons"]
    lr = params["lr"]
    epochs = params["epochs"]
    batch_size = params["batch size"]
    print(f"hyperparams: hidden neurons={hidden_neurons} lr={lr} epochs={epochs} batch size={1}:")
    for k, (fold_X_train,fold_y_train,fold_X_val,fold_y_val) in enumerate(zip(X_train, y_train,X_val,y_val)):
        print("Fold: ", k+1)
        simple_nn=simpleNN.SimpleNN(len(fold_X_train[0]),hidden_neurons)
        start = time.time()
        fold_precision,fold_recall,fold_fscore=simpleNN.train_simpleNN(
                    np.array(fold_X_train),np.array(fold_X_val), 
                    np.array(fold_y_train),np.array(fold_y_val),
                    loss_fn=torch.nn.BCELoss(),
                    model = simple_nn, 
                    optimizer=torch.optim.SGD(simple_nn.parameters(),lr=lr),
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=True
                    )
        stop = time.time()
        print(f"precision={fold_precision},recall={fold_recall},score={fold_fscore}")
        results_df = pd.concat(
            [
                results_df,
                pd.DataFrame(
                    {
                        "batch size": batch_size,
                        "hidden neurons": params["hidden_neurons"],
                        "learning rate": params["lr"],
                        "optimizer": "SGD",
                        "epochs": params["epochs"],
                        "Fold": k,
                        "Loss":"BCELoss",
                        "F1 Score": fold_fscore,
                        "Recall": fold_recall,
                        "Precision": fold_precision,
                        "Time": stop - start,

                    },
                    index=[0],
                ),
            ],
            ignore_index=True,
        )
    # save results every time we finish a fold in case of crash
    results_df.to_csv(RES_DIR + "results-SimpleNN-w2v.csv", index=False)
results_df

Training a simple NN
hyperparams: hidden neurons=20 lr=0.001 epochs=10 batch size=1:
Fold:  1
'Epoch 1 -------------------------------'
'loss: 0.787666  [    1/23944]'
'loss: 0.512426  [ 1001/23944]'
'loss: 0.837770  [ 2001/23944]'
'loss: 0.618263  [ 3001/23944]'
'loss: 0.893635  [ 4001/23944]'
'loss: 0.582270  [ 5001/23944]'
'loss: 0.842003  [ 6001/23944]'
'loss: 0.580999  [ 7001/23944]'
'loss: 0.806035  [ 8001/23944]'
'loss: 0.540476  [ 9001/23944]'
'loss: 0.507259  [10001/23944]'
'loss: 0.855323  [11001/23944]'
'loss: 0.566647  [12001/23944]'
'loss: 0.573106  [13001/23944]'
'loss: 0.572137  [14001/23944]'
'loss: 0.569592  [15001/23944]'
'loss: 0.595655  [16001/23944]'
'loss: 0.779038  [17001/23944]'
'loss: 0.800200  [18001/23944]'
'loss: 0.598819  [19001/23944]'
'loss: 0.777898  [20001/23944]'
'loss: 0.562434  [21001/23944]'
'loss: 0.596237  [22001/23944]'
'loss: 0.826376  [23001/23944]'
'Epoch 2 -------------------------------'
'loss: 0.755988  [    1/23944]'
'loss: 0.491937  [ 100

C:\Users\mirda\AppData\Local\Temp\ipykernel_6244\1705577380.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat(


'loss: 0.599311  [ 1001/23945]'
'loss: 0.595096  [ 2001/23945]'
'loss: 0.619727  [ 3001/23945]'
'loss: 0.526154  [ 4001/23945]'
'loss: 0.603801  [ 5001/23945]'
'loss: 0.565016  [ 6001/23945]'
'loss: 0.806231  [ 7001/23945]'
'loss: 0.582135  [ 8001/23945]'
'loss: 0.530209  [ 9001/23945]'
'loss: 0.872991  [10001/23945]'
'loss: 0.802712  [11001/23945]'
'loss: 0.575058  [12001/23945]'
'loss: 0.790525  [13001/23945]'
'loss: 0.545663  [14001/23945]'
'loss: 0.851917  [15001/23945]'
'loss: 0.552868  [16001/23945]'
'loss: 0.586686  [17001/23945]'
'loss: 0.743727  [18001/23945]'
'loss: 0.567420  [19001/23945]'
'loss: 0.590086  [20001/23945]'
'loss: 0.585620  [21001/23945]'
'loss: 0.570367  [22001/23945]'
'loss: 0.814386  [23001/23945]'
'Epoch 2 -------------------------------'
'loss: 0.820355  [    1/23945]'
'loss: 0.506294  [ 1001/23945]'
'loss: 0.571237  [ 2001/23945]'
'loss: 0.644500  [ 3001/23945]'
'loss: 0.529440  [ 4001/23945]'
'loss: 0.593218  [ 5001/23945]'
'loss: 0.545082  [ 6001/23945]

C:\Users\mirda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


precision=0.280654861343134,recall=0.5,score=0.35951209073400386
Fold:  5
'Epoch 1 -------------------------------'
'loss: 0.687144  [    5/23945]'
'loss: 0.591038  [ 5005/23945]'
'loss: 0.675819  [10005/23945]'
'loss: 0.835536  [15005/23945]'
'loss: 0.627834  [20005/23945]'
'Epoch 2 -------------------------------'
'loss: 0.673537  [    5/23945]'
'loss: 0.571631  [ 5005/23945]'
'loss: 0.674936  [10005/23945]'
'loss: 0.834914  [15005/23945]'
'loss: 0.627958  [20005/23945]'
'Epoch 3 -------------------------------'
'loss: 0.671070  [    5/23945]'
'loss: 0.570021  [ 5005/23945]'
'loss: 0.674227  [10005/23945]'
'loss: 0.833124  [15005/23945]'
'loss: 0.628202  [20005/23945]'
'Epoch 4 -------------------------------'
'loss: 0.668436  [    5/23945]'
'loss: 0.568275  [ 5005/23945]'
'loss: 0.673480  [10005/23945]'
'loss: 0.831170  [15005/23945]'
'loss: 0.628485  [20005/23945]'
'Epoch 5 -------------------------------'
'loss: 0.665540  [    5/23945]'
'loss: 0.566329  [ 5005/23945]'
'loss: 0.672

...

C:\Users\mirda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


precision=0.280654861343134,recall=0.5,score=0.35951209073400386
Fold:  5
'Epoch 1 -------------------------------'
'loss: 0.670082  [   10/23945]'
'loss: 0.737247  [10010/23945]'
'loss: 0.622306  [20010/23945]'
'Epoch 2 -------------------------------'
'loss: 0.672209  [   10/23945]'
'loss: 0.729355  [10010/23945]'
'loss: 0.624184  [20010/23945]'
'Epoch 3 -------------------------------'
'loss: 0.671776  [   10/23945]'
'loss: 0.728944  [10010/23945]'
'loss: 0.623794  [20010/23945]'
'Epoch 4 -------------------------------'
'loss: 0.671127  [   10/23945]'
'loss: 0.728729  [10010/23945]'
'loss: 0.623331  [20010/23945]'
'Epoch 5 -------------------------------'
'loss: 0.670454  [   10/23945]'
'loss: 0.728511  [10010/23945]'
'loss: 0.622854  [20010/23945]'
'Epoch 6 -------------------------------'
'loss: 0.669757  [   10/23945]'
'loss: 0.728283  [10010/23945]'
'loss: 0.622359  [20010/23945]'
'Epoch 7 -------------------------------'
'loss: 0.669031  [   10/23945]'
'loss: 0.728042  [10010/

C:\Users\mirda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


precision=0.2817407283661878,recall=0.5,score=0.36040175232396626
hyperparams: hidden neurons=20 lr=0.01 epochs=10 batch size=1:
Fold:  1
'Epoch 1 -------------------------------'
'loss: 0.698426  [   10/23944]'
'loss: 0.639022  [10010/23944]'
'loss: 0.662977  [20010/23944]'
'Epoch 2 -------------------------------'
'loss: 0.694619  [   10/23944]'
'loss: 0.645010  [10010/23944]'
'loss: 0.661359  [20010/23944]'
'Epoch 3 -------------------------------'
'loss: 0.681764  [   10/23944]'
'loss: 0.655678  [10010/23944]'
'loss: 0.658096  [20010/23944]'
'Epoch 4 -------------------------------'
'loss: 0.661067  [   10/23944]'
'loss: 0.670273  [10010/23944]'
'loss: 0.651079  [20010/23944]'
'Epoch 5 -------------------------------'
'loss: 0.639367  [   10/23944]'
'loss: 0.679878  [10010/23944]'
'loss: 0.638881  [20010/23944]'
'Epoch 6 -------------------------------'
'loss: 0.626049  [   10/23944]'
'loss: 0.677157  [10010/23944]'
'loss: 0.623465  [20010/23944]'
'Epoch 7 -------------------------

,hidden neurons,optimizer,batch size,epochs,F1 Score,Recall,Precision,learning rate,Fold,Loss,Time
0,20,SGD,1,10,0.688944,0.686865,0.706570,0.001,0.0,BCELoss,276.920604
1,20,SGD,1,10,0.682347,0.681462,0.710435,0.001,1.0,BCELoss,287.070978
2,20,SGD,1,10,0.692398,0.691039,0.712630,0.001,2.0,BCELoss,254.965207
3,20,SGD,1,10,0.688630,0.687488,0.711904,0.001,3.0,BCELoss,267.776694
4,20,SGD,1,10,0.685509,0.684588,0.712495,0.001,4.0,BCELoss,265.067430
...,...,...,...,...,...,...,...,...,...,...,...
85,20,SGD,10,20,0.726282,0.724445,0.729976,0.100,0.0,BCELoss,37.562799
86,20,SGD,10,20,0.722237,0.719120,0.742707,0.100,1.0,BCELoss,36.362339
87,20,SGD,10,20,0.726972,0.724943,0.732754,0.100,2.0,BCELoss,37.643547
88,20,SGD,10,20,0.729332,0.728822,0.729975,0.100,3.0,BCELoss,40.083056


## Fasttext embeddings

In [7]:
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess

In [8]:
X_train, y_train, X_val, y_val, _, _ = datautils.split_kfold_dataset(PATH)

for idx, (tr_fold, val_fold) in enumerate(zip(X_train, X_val)):
    word_vectors = KeyedVectors.load(EMBED_DIR+f'w2v-fold{idx}.kv', mmap='r')

    tr_fold = list(map(simple_preprocess, tr_fold))
    val_fold = list(map(simple_preprocess, val_fold))

    X_train[idx] = datautils.documents_vector_wv(tr_fold,word_vectors)
    X_val[idx] = datautils.documents_vector_wv(val_fold,word_vectors)

In [9]:
#train simple NN here
# train simple NN here

import numpy as np


grid = list(ParameterGrid(
    {
        "hidden_neurons": [20], 
        "lr": [1e-3, 1e-2, 1e-1],
        "epochs": [10,20],
        "batch size": [1,5,10]
    }
))
# train simple NN here
results_df = pd.DataFrame(
    columns=["hidden neurons","optimizer","batch size" ,"epochs", "F1 Score", "Recall", "Precision"]
)
print(f"Training a simple NN")
for params in grid:
    hidden_neurons = params["hidden_neurons"]
    lr = params["lr"]
    epochs = params["epochs"]
    batch_size = params["batch size"]
    print(f"hyperparams: hidden neurons={hidden_neurons} lr={lr} epochs={epochs} batch size={1}:")
    for k, (fold_X_train,fold_y_train,fold_X_val,fold_y_val) in enumerate(zip(X_train, y_train,X_val,y_val)):
        print("Fold: ", k+1)
        simple_nn=simpleNN.SimpleNN(len(fold_X_train[0]),hidden_neurons)
        start = time.time()
        fold_precision,fold_recall,fold_fscore=simpleNN.train_simpleNN(
                    np.array(fold_X_train),np.array(fold_X_val), 
                    np.array(fold_y_train),np.array(fold_y_val),
                    loss_fn=torch.nn.BCELoss(),
                    model = simple_nn, 
                    optimizer=torch.optim.SGD(simple_nn.parameters(),lr=lr),
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=True
                    )
        stop = time.time()
        print(f"precision={fold_precision},recall={fold_recall},score={fold_fscore}")
        results_df = pd.concat(
            [
                results_df,
                pd.DataFrame(
                    {
                        "batch size": batch_size,
                        "hidden neurons": params["hidden_neurons"],
                        "learning rate": params["lr"],
                        "optimizer": "SGD",
                        "epochs": params["epochs"],
                        "Fold": k,
                        "Loss":"BCELoss",
                        "F1 Score": fold_fscore,
                        "Recall": fold_recall,
                        "Precision": fold_precision,
                        "Time": stop - start,

                    },
                    index=[0],
                ),
            ],
            ignore_index=True,
        )
    # save results every time we finish a fold in case of crash
    results_df.to_csv(RES_DIR + "results-SimpleNN-ftx.csv", index=False)
results_df

Training a simple NN
hyperparams: hidden neurons=20 lr=0.001 epochs=10 batch size=1:
Fold:  1
'Epoch 1 -------------------------------'
'loss: 0.799870  [    1/23944]'
'loss: 0.520153  [ 1001/23944]'
'loss: 0.822397  [ 2001/23944]'
'loss: 0.620308  [ 3001/23944]'
'loss: 0.906640  [ 4001/23944]'
'loss: 0.577638  [ 5001/23944]'
'loss: 0.840158  [ 6001/23944]'
'loss: 0.584462  [ 7001/23944]'
'loss: 0.808823  [ 8001/23944]'
'loss: 0.530591  [ 9001/23944]'
'loss: 0.507246  [10001/23944]'
'loss: 0.850215  [11001/23944]'
'loss: 0.565633  [12001/23944]'
'loss: 0.583082  [13001/23944]'
'loss: 0.570718  [14001/23944]'
'loss: 0.563142  [15001/23944]'
'loss: 0.597833  [16001/23944]'
'loss: 0.802931  [17001/23944]'
'loss: 0.808031  [18001/23944]'
'loss: 0.594617  [19001/23944]'
'loss: 0.760148  [20001/23944]'
'loss: 0.572820  [21001/23944]'
'loss: 0.593706  [22001/23944]'
'loss: 0.810876  [23001/23944]'
'Epoch 2 -------------------------------'
'loss: 0.784820  [    1/23944]'
'loss: 0.495741  [ 100

C:\Users\mirda\AppData\Local\Temp\ipykernel_6244\1050298156.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat(


'loss: 0.599322  [ 1001/23945]'
'loss: 0.576338  [ 2001/23945]'
'loss: 0.636930  [ 3001/23945]'
'loss: 0.520183  [ 4001/23945]'
'loss: 0.600617  [ 5001/23945]'
'loss: 0.562494  [ 6001/23945]'
'loss: 0.777160  [ 7001/23945]'
'loss: 0.575318  [ 8001/23945]'
'loss: 0.538890  [ 9001/23945]'
'loss: 0.838245  [10001/23945]'
'loss: 0.811206  [11001/23945]'
'loss: 0.569169  [12001/23945]'
'loss: 0.802464  [13001/23945]'
'loss: 0.565684  [14001/23945]'
'loss: 0.848888  [15001/23945]'
'loss: 0.569644  [16001/23945]'
'loss: 0.575581  [17001/23945]'
'loss: 0.743226  [18001/23945]'
'loss: 0.587941  [19001/23945]'
'loss: 0.583146  [20001/23945]'
'loss: 0.589774  [21001/23945]'
'loss: 0.588360  [22001/23945]'
'loss: 0.790321  [23001/23945]'
'Epoch 2 -------------------------------'
'loss: 0.803465  [    1/23945]'
'loss: 0.507956  [ 1001/23945]'
'loss: 0.545081  [ 2001/23945]'
'loss: 0.668394  [ 3001/23945]'
'loss: 0.522760  [ 4001/23945]'
'loss: 0.588479  [ 5001/23945]'
'loss: 0.535658  [ 6001/23945]

C:\Users\mirda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


precision=0.28403206948388177,recall=0.5,score=0.3622709842351939
Fold:  2
'Epoch 1 -------------------------------'
'loss: 0.712421  [   10/23945]'
'loss: 0.700373  [10010/23945]'
'loss: 0.746720  [20010/23945]'
'Epoch 2 -------------------------------'
'loss: 0.728859  [   10/23945]'
'loss: 0.701417  [10010/23945]'
'loss: 0.747134  [20010/23945]'
'Epoch 3 -------------------------------'
'loss: 0.729142  [   10/23945]'
'loss: 0.701061  [10010/23945]'
'loss: 0.746222  [20010/23945]'
'Epoch 4 -------------------------------'
'loss: 0.728969  [   10/23945]'
'loss: 0.700666  [10010/23945]'
'loss: 0.745256  [20010/23945]'
'Epoch 5 -------------------------------'
'loss: 0.728787  [   10/23945]'
'loss: 0.700264  [10010/23945]'
'loss: 0.744261  [20010/23945]'
'Epoch 6 -------------------------------'
'loss: 0.728603  [   10/23945]'
'loss: 0.699855  [10010/23945]'
'loss: 0.743229  [20010/23945]'
'Epoch 7 -------------------------------'
'loss: 0.728416  [   10/23945]'
'loss: 0.699434  [10010

...

C:\Users\mirda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


precision=0.2817407283661878,recall=0.5,score=0.36040175232396626
hyperparams: hidden neurons=20 lr=0.01 epochs=10 batch size=1:
Fold:  1
'Epoch 1 -------------------------------'
'loss: 0.691450  [   10/23944]'
'loss: 0.628742  [10010/23944]'
'loss: 0.662391  [20010/23944]'
'Epoch 2 -------------------------------'
'loss: 0.690496  [   10/23944]'
'loss: 0.631654  [10010/23944]'
'loss: 0.661543  [20010/23944]'
'Epoch 3 -------------------------------'
'loss: 0.677461  [   10/23944]'
'loss: 0.637754  [10010/23944]'
'loss: 0.659282  [20010/23944]'
'Epoch 4 -------------------------------'
'loss: 0.656123  [   10/23944]'
'loss: 0.648035  [10010/23944]'
'loss: 0.653560  [20010/23944]'
'Epoch 5 -------------------------------'
'loss: 0.634557  [   10/23944]'
'loss: 0.656163  [10010/23944]'
'loss: 0.642435  [20010/23944]'
'Epoch 6 -------------------------------'
'loss: 0.622662  [   10/23944]'
'loss: 0.654680  [10010/23944]'
'loss: 0.627469  [20010/23944]'
'Epoch 7 -------------------------

C:\Users\mirda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


precision=0.2817407283661878,recall=0.5,score=0.36040175232396626
hyperparams: hidden neurons=20 lr=0.01 epochs=20 batch size=1:
Fold:  1
'Epoch 1 -------------------------------'
'loss: 0.690535  [   10/23944]'
'loss: 0.625649  [10010/23944]'
'loss: 0.659053  [20010/23944]'
'Epoch 2 -------------------------------'
'loss: 0.691410  [   10/23944]'
'loss: 0.628165  [10010/23944]'
'loss: 0.657007  [20010/23944]'
'Epoch 3 -------------------------------'
'loss: 0.681276  [   10/23944]'
'loss: 0.632343  [10010/23944]'
'loss: 0.653496  [20010/23944]'
'Epoch 4 -------------------------------'
'loss: 0.662850  [   10/23944]'
'loss: 0.640969  [10010/23944]'
'loss: 0.648074  [20010/23944]'
'Epoch 5 -------------------------------'
'loss: 0.640166  [   10/23944]'
'loss: 0.651229  [10010/23944]'
'loss: 0.639732  [20010/23944]'
'Epoch 6 -------------------------------'
'loss: 0.624706  [   10/23944]'
'loss: 0.654035  [10010/23944]'
'loss: 0.628092  [20010/23944]'
'Epoch 7 -------------------------

,hidden neurons,optimizer,batch size,epochs,F1 Score,Recall,Precision,learning rate,Fold,Loss,Time
0,20,SGD,1,10,0.689357,0.687228,0.705792,0.001,0.0,BCELoss,178.141809
1,20,SGD,1,10,0.684316,0.683296,0.712171,0.001,1.0,BCELoss,181.531230
2,20,SGD,1,10,0.691593,0.690280,0.712003,0.001,2.0,BCELoss,177.530042
3,20,SGD,1,10,0.690599,0.689399,0.714418,0.001,3.0,BCELoss,184.047512
4,20,SGD,1,10,0.683118,0.682478,0.711790,0.001,4.0,BCELoss,170.828651
...,...,...,...,...,...,...,...,...,...,...,...
85,20,SGD,10,20,0.726662,0.725085,0.729515,0.100,0.0,BCELoss,38.355341
86,20,SGD,10,20,0.723759,0.720583,0.744514,0.100,1.0,BCELoss,38.136704
87,20,SGD,10,20,0.726316,0.724305,0.732024,0.100,2.0,BCELoss,37.166053
88,20,SGD,10,20,0.730187,0.729934,0.730470,0.100,3.0,BCELoss,36.351763


# Metrics
## TF-IDF


In [40]:
# load previsouslt computed metrics from their csv file
results_df = pd.read_csv(RES_DIR + "results-SimpleNN-tfidf.csv")
#compute metrics for each hyperparameter configuration
meanf1=results_df.groupby(["hidden neurons","optimizer","batch size","epochs","learning rate","Loss"]).mean().drop(columns="Fold").sort_values("F1 Score",ascending=False)
meanf1

F1 Score  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       1          10     0.010         BCELoss  0.775620   
                         10         10     0.100         BCELoss  0.774712   
                         5          10     0.100         BCELoss  0.771189   
                         10         20     0.100         BCELoss  0.770044   
                         1          20     0.010         BCELoss  0.768006   
                         5          20     0.100         BCELoss  0.747484   
                         1          20     0.100         BCELoss  0.735161   
                                    10     0.100         BCELoss  0.733548   
                         5          20     0.010         BCELoss  0.593341   
                         10         10     0.010         BCELoss  0.360460   
                                    20     0.010         BCELoss  0.360460   
                                           0.001         BCELoss  0.360460   
                         1          10     0.001         BCELoss  0.360460   
                         10         10     0.001         BCELoss  0.360460   
                         5          10     0.010         BCELoss  0.360460   
                                           0.001         BCELoss  0.360460   
                         1          20     0.001         BCELoss  0.360460   
                         5          20     0.001         BCELoss  0.360460   

                                                                    Recall  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       1          10     0.010         BCELoss  0.774673   
                         10         10     0.100         BCELoss  0.773142   
                         5          10     0.100         BCELoss  0.771877   
                         10         20     0.100         BCELoss  0.770924   
                         1          20     0.010         BCELoss  0.769582   
                         5          20     0.100         BCELoss  0.749251   
                         1          20     0.100         BCELoss  0.738003   
                                    10     0.100         BCELoss  0.736520   
                         5          20     0.010         BCELoss  0.626152   
                         10         10     0.010         BCELoss  0.500000   
                                    20     0.010         BCELoss  0.500000   
                                           0.001         BCELoss  0.500000   
                         1          10     0.001         BCELoss  0.500000   
                         10         10     0.001         BCELoss  0.500000   
                         5          10     0.010         BCELoss  0.500000   
                                           0.001         BCELoss  0.500000   
                         1          20     0.001         BCELoss  0.500000   
                         5          20     0.001         BCELoss  0.500000   

                                                                  Precision  \
hidden neurons optimizer batch size epochs learning rate Loss                 
20             SGD       1          10     0.010         BCELoss   0.781461   
                         10         10     0.100         BCELoss   0.782109   
                         5          10     0.100         BCELoss   0.774172   
                         10         20     0.100         BCELoss   0.771934   
                         1          20     0.010         BCELoss   0.769794   
                         5          20     0.100         BCELoss   0.751171   
                         1          20     0.100         BCELoss   0.735303   
                                    10     0.100         BCELoss   0.741629   
                         5          20     0.010         BCELoss   0.763933   
                         10         10     0.010         BCELoss   0.281815

In [48]:
stdf1=results_df.groupby(["hidden neurons","optimizer","batch size","epochs","learning rate","Loss"]).std().drop(columns="Fold").sort_values("F1 Score",ascending=False)
stdf1

F1 Score  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       5          20     0.001         BCELoss  0.027478   
                         1          20     0.100         BCELoss  0.014033   
                                    10     0.100         BCELoss  0.010054   
                         5          10     0.001         BCELoss  0.007423   
                         1          20     0.010         BCELoss  0.005660   
                         5          10     0.010         BCELoss  0.005656   
                                    20     0.100         BCELoss  0.005498   
                         1          10     0.010         BCELoss  0.005323   
                         5          20     0.010         BCELoss  0.005277   
                                    10     0.100         BCELoss  0.005203   
                         10         20     0.100         BCELoss  0.004729   
                                    10     0.100         BCELoss  0.004714   
                         1          10     0.001         BCELoss  0.003803   
                         10         10     0.010         BCELoss  0.003678   
                         1          20     0.001         BCELoss  0.003147   
                         10         20     0.010         BCELoss  0.002758   
                                           0.001         BCELoss  0.002242   
                                    10     0.001         BCELoss  0.001373   

                                                                    Recall  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       5          20     0.001         BCELoss  0.015756   
                         1          20     0.100         BCELoss  0.013169   
                                    10     0.100         BCELoss  0.007703   
                         5          10     0.001         BCELoss  0.002736   
                         1          20     0.010         BCELoss  0.006299   
                         5          10     0.010         BCELoss  0.005468   
                                    20     0.100         BCELoss  0.005049   
                         1          10     0.010         BCELoss  0.005864   
                         5          20     0.010         BCELoss  0.005211   
                                    10     0.100         BCELoss  0.005715   
                         10         20     0.100         BCELoss  0.005560   
                                    10     0.100         BCELoss  0.005359   
                         1          10     0.001         BCELoss  0.003554   
                         10         10     0.010         BCELoss  0.003351   
                         1          20     0.001         BCELoss  0.003030   
                         10         20     0.010         BCELoss  0.002734   
                                           0.001         BCELoss  0.001072   
                                    10     0.001         BCELoss  0.000000   

                                                                  Precision  \
hidden neurons optimizer batch size epochs learning rate Loss                 
20             SGD       5          20     0.001         BCELoss   0.006510   
                         1          20     0.100         BCELoss   0.012674   
                                    10     0.100         BCELoss   0.013160   
                         5          10     0.001         BCELoss   0.210888   
                         1          20     0.010         BCELoss   0.005815   
                         5          10     0.010         BCELoss   0.003518   
                                    20     0.100         BCELoss   0.010006   
                         1          10     0.010         BCELoss   0.006747   
                         5          20     0.010         BCELoss   0.004738   
                                    10     0.100         BCELoss   0.007437

## W2V

In [46]:
# load previsouslt computed metrics from their csv file
results_df = pd.read_csv(RES_DIR + "results-SimpleNN-w2v.csv")
#compute metrics for each hyperparameter configuration
results_df.groupby(["hidden neurons","optimizer","batch size","epochs","learning rate","Loss"]).mean().drop(columns="Fold").sort_values("F1 Score",ascending=False)


F1 Score  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       5          20     0.100         BCELoss  0.728184   
                                    10     0.100         BCELoss  0.724430   
                         10         20     0.100         BCELoss  0.724426   
                         1          10     0.100         BCELoss  0.723017   
                                    20     0.010         BCELoss  0.722919   
                                           0.100         BCELoss  0.719601   
                                    10     0.010         BCELoss  0.719374   
                         10         10     0.100         BCELoss  0.717793   
                         5          20     0.010         BCELoss  0.716703   
                                    10     0.010         BCELoss  0.711248   
                         10         20     0.010         BCELoss  0.710338   
                         1          20     0.001         BCELoss  0.709552   
                                    10     0.001         BCELoss  0.687565   
                         10         10     0.010         BCELoss  0.685581   
                         5          20     0.001         BCELoss  0.597965   
                         10         20     0.001         BCELoss  0.365792   
                         5          10     0.001         BCELoss  0.365765   
                         10         10     0.001         BCELoss  0.360460   

                                                                    Recall  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       5          20     0.100         BCELoss  0.726813   
                                    10     0.100         BCELoss  0.722985   
                         10         20     0.100         BCELoss  0.722384   
                         1          10     0.100         BCELoss  0.723344   
                                    20     0.010         BCELoss  0.721182   
                                           0.100         BCELoss  0.719682   
                                    10     0.010         BCELoss  0.717792   
                         10         10     0.100         BCELoss  0.715892   
                         5          20     0.010         BCELoss  0.714225   
                                    10     0.010         BCELoss  0.708927   
                         10         20     0.010         BCELoss  0.707887   
                         1          20     0.001         BCELoss  0.707154   
                                    10     0.001         BCELoss  0.686289   
                         10         10     0.010         BCELoss  0.684415   
                         5          20     0.001         BCELoss  0.618824   
                         10         20     0.001         BCELoss  0.502388   
                         5          10     0.001         BCELoss  0.502319   
                         10         10     0.001         BCELoss  0.500000   

                                                                  Precision  \
hidden neurons optimizer batch size epochs learning rate Loss                 
20             SGD       5          20     0.100         BCELoss   0.735766   
                                    10     0.100         BCELoss   0.731858   
                         10         20     0.100         BCELoss   0.733662   
                         1          10     0.100         BCELoss   0.730721   
                                    20     0.010         BCELoss   0.732025   
                                           0.100         BCELoss   0.731742   
                                    10     0.010         BCELoss   0.728001   
                         10         10     0.100         BCELoss   0.729261   
                         5          20     0.010         BCELoss   0.731016   
                                    10     0.010         BCELoss   0.726724

In [47]:
results_df.groupby(["hidden neurons","optimizer","batch size","epochs","learning rate","Loss"]).std().drop(columns="Fold").sort_values("F1 Score",ascending=False)

F1 Score  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       5          20     0.001         BCELoss  0.027478   
                         1          20     0.100         BCELoss  0.014033   
                                    10     0.100         BCELoss  0.010054   
                         5          10     0.001         BCELoss  0.007423   
                         1          20     0.010         BCELoss  0.005660   
                         5          10     0.010         BCELoss  0.005656   
                                    20     0.100         BCELoss  0.005498   
                         1          10     0.010         BCELoss  0.005323   
                         5          20     0.010         BCELoss  0.005277   
                                    10     0.100         BCELoss  0.005203   
                         10         20     0.100         BCELoss  0.004729   
                                    10     0.100         BCELoss  0.004714   
                         1          10     0.001         BCELoss  0.003803   
                         10         10     0.010         BCELoss  0.003678   
                         1          20     0.001         BCELoss  0.003147   
                         10         20     0.010         BCELoss  0.002758   
                                           0.001         BCELoss  0.002242   
                                    10     0.001         BCELoss  0.001373   

                                                                    Recall  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       5          20     0.001         BCELoss  0.015756   
                         1          20     0.100         BCELoss  0.013169   
                                    10     0.100         BCELoss  0.007703   
                         5          10     0.001         BCELoss  0.002736   
                         1          20     0.010         BCELoss  0.006299   
                         5          10     0.010         BCELoss  0.005468   
                                    20     0.100         BCELoss  0.005049   
                         1          10     0.010         BCELoss  0.005864   
                         5          20     0.010         BCELoss  0.005211   
                                    10     0.100         BCELoss  0.005715   
                         10         20     0.100         BCELoss  0.005560   
                                    10     0.100         BCELoss  0.005359   
                         1          10     0.001         BCELoss  0.003554   
                         10         10     0.010         BCELoss  0.003351   
                         1          20     0.001         BCELoss  0.003030   
                         10         20     0.010         BCELoss  0.002734   
                                           0.001         BCELoss  0.001072   
                                    10     0.001         BCELoss  0.000000   

                                                                  Precision  \
hidden neurons optimizer batch size epochs learning rate Loss                 
20             SGD       5          20     0.001         BCELoss   0.006510   
                         1          20     0.100         BCELoss   0.012674   
                                    10     0.100         BCELoss   0.013160   
                         5          10     0.001         BCELoss   0.210888   
                         1          20     0.010         BCELoss   0.005815   
                         5          10     0.010         BCELoss   0.003518   
                                    20     0.100         BCELoss   0.010006   
                         1          10     0.010         BCELoss   0.006747   
                         5          20     0.010         BCELoss   0.004738   
                                    10     0.100         BCELoss   0.007437

## FTX

In [39]:
# load previsouslt computed metrics from their csv file
results_df = pd.read_csv(RES_DIR + "results-SimpleNN-ftx.csv")
#compute metrics for each hyperparameter configuration
results_df.groupby(["hidden neurons","optimizer","batch size","epochs","learning rate","Loss"]).mean().drop(columns="Fold").sort_values("F1 Score",ascending=False)

F1 Score  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       5          20     0.100         BCELoss  0.727742   
                         1          20     0.100         BCELoss  0.725858   
                         10         20     0.100         BCELoss  0.725483   
                         1          20     0.010         BCELoss  0.725206   
                         5          10     0.100         BCELoss  0.722603   
                         1          10     0.010         BCELoss  0.719741   
                                           0.100         BCELoss  0.719672   
                         10         10     0.100         BCELoss  0.718418   
                         5          20     0.010         BCELoss  0.716227   
                                    10     0.010         BCELoss  0.711670   
                         1          20     0.001         BCELoss  0.709642   
                         10         20     0.010         BCELoss  0.709400   
                         1          10     0.001         BCELoss  0.687797   
                         10         10     0.010         BCELoss  0.687614   
                         5          20     0.001         BCELoss  0.587085   
                                    10     0.001         BCELoss  0.373034   
                         10         20     0.001         BCELoss  0.370161   
                                    10     0.001         BCELoss  0.360460   

                                                                    Recall  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       5          20     0.100         BCELoss  0.726328   
                         1          20     0.100         BCELoss  0.725400   
                         10         20     0.100         BCELoss  0.723520   
                         1          20     0.010         BCELoss  0.723348   
                         5          10     0.100         BCELoss  0.721061   
                         1          10     0.010         BCELoss  0.718141   
                                           0.100         BCELoss  0.720282   
                         10         10     0.100         BCELoss  0.716750   
                         5          20     0.010         BCELoss  0.713756   
                                    10     0.010         BCELoss  0.709333   
                         1          20     0.001         BCELoss  0.707243   
                         10         20     0.010         BCELoss  0.706993   
                         1          10     0.001         BCELoss  0.686536   
                         10         10     0.010         BCELoss  0.686325   
                         5          20     0.001         BCELoss  0.613097   
                                    10     0.001         BCELoss  0.505515   
                         10         20     0.001         BCELoss  0.504253   
                                    10     0.001         BCELoss  0.500000   

                                                                  Precision  \
hidden neurons optimizer batch size epochs learning rate Loss                 
20             SGD       5          20     0.100         BCELoss   0.735388   
                         1          20     0.100         BCELoss   0.733964   
                         10         20     0.100         BCELoss   0.734369   
                         1          20     0.010         BCELoss   0.734293   
                         5          10     0.100         BCELoss   0.730780   
                         1          10     0.010         BCELoss   0.728611   
                                           0.100         BCELoss   0.729534   
                         10         10     0.100         BCELoss   0.728830   
                         5          20     0.010         BCELoss   0.730654   
                                    10     0.010         BCELoss   0.726605

In [49]:
results_df.groupby(["hidden neurons","optimizer","batch size","epochs","learning rate","Loss"]).std().drop(columns="Fold").sort_values("F1 Score",ascending=False)

F1 Score  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       5          20     0.001         BCELoss  0.027478   
                         1          20     0.100         BCELoss  0.014033   
                                    10     0.100         BCELoss  0.010054   
                         5          10     0.001         BCELoss  0.007423   
                         1          20     0.010         BCELoss  0.005660   
                         5          10     0.010         BCELoss  0.005656   
                                    20     0.100         BCELoss  0.005498   
                         1          10     0.010         BCELoss  0.005323   
                         5          20     0.010         BCELoss  0.005277   
                                    10     0.100         BCELoss  0.005203   
                         10         20     0.100         BCELoss  0.004729   
                                    10     0.100         BCELoss  0.004714   
                         1          10     0.001         BCELoss  0.003803   
                         10         10     0.010         BCELoss  0.003678   
                         1          20     0.001         BCELoss  0.003147   
                         10         20     0.010         BCELoss  0.002758   
                                           0.001         BCELoss  0.002242   
                                    10     0.001         BCELoss  0.001373   

                                                                    Recall  \
hidden neurons optimizer batch size epochs learning rate Loss                
20             SGD       5          20     0.001         BCELoss  0.015756   
                         1          20     0.100         BCELoss  0.013169   
                                    10     0.100         BCELoss  0.007703   
                         5          10     0.001         BCELoss  0.002736   
                         1          20     0.010         BCELoss  0.006299   
                         5          10     0.010         BCELoss  0.005468   
                                    20     0.100         BCELoss  0.005049   
                         1          10     0.010         BCELoss  0.005864   
                         5          20     0.010         BCELoss  0.005211   
                                    10     0.100         BCELoss  0.005715   
                         10         20     0.100         BCELoss  0.005560   
                                    10     0.100         BCELoss  0.005359   
                         1          10     0.001         BCELoss  0.003554   
                         10         10     0.010         BCELoss  0.003351   
                         1          20     0.001         BCELoss  0.003030   
                         10         20     0.010         BCELoss  0.002734   
                                           0.001         BCELoss  0.001072   
                                    10     0.001         BCELoss  0.000000   

                                                                  Precision  \
hidden neurons optimizer batch size epochs learning rate Loss                 
20             SGD       5          20     0.001         BCELoss   0.006510   
                         1          20     0.100         BCELoss   0.012674   
                                    10     0.100         BCELoss   0.013160   
                         5          10     0.001         BCELoss   0.210888   
                         1          20     0.010         BCELoss   0.005815   
                         5          10     0.010         BCELoss   0.003518   
                                    20     0.100         BCELoss   0.010006   
                         1          10     0.010         BCELoss   0.006747   
                         5          20     0.010         BCELoss   0.004738   
                                    10     0.100         BCELoss   0.007437